In [27]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import math
import time
import hydra
import logging
import argparse


from numpy import random
from omegaconf import DictConfig, OmegaConf, ValidationError



from core.pgm import PGM
from utils.weather import Weather
from core.vehicle_manager import VehicleManager
from core.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints
from shcemas.pgm_schema import PGMSchema

In [28]:
from hydra.core.config_store import ConfigStore
from shcemas.pgm_schema import PGMModel


cs = ConfigStore.instance()
cs.store(name="base_config", node=PGMSchema)
cs.store(group="model", name="base_model", node=PGMModel)
with hydra.initialize(version_base=None, config_path="conf/pgm"):
    cfg: PGMSchema = hydra.compose(config_name="config")
    # print(cfg)
    # print(OmegaConf.to_yaml(cfg.model))
    pgm = PGM(cfg)
    pgm.print_cpd('T')

+----------+------------+---------------+---------------+
| R        | R(NO_RAIN) | R(LIGHT_RAIN) | R(HEAVY_RAIN) |
+----------+------------+---------------+---------------+
| T(LOW)   | 0.1        | 0.4           | 0.9           |
+----------+------------+---------------+---------------+
| T(HEAVY) | 0.9        | 0.6           | 0.1           |
+----------+------------+---------------+---------------+


In [29]:
states = pgm.get_states()

In [30]:
states.Rain.NO_RAIN

<R.NO_RAIN: 1>

In [5]:
res = pgm.predict_dist(['T', 'S'], evidence={'R': 'NO_RAIN'})
print(res)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+---------+----------+------------+
| S       | T        |   phi(S,T) |
+=========+==========+============+
| S(LOW)  | T(LOW)   |     0.0100 |
+---------+----------+------------+
| S(LOW)  | T(HEAVY) |     0.0900 |
+---------+----------+------------+
| S(HIGH) | T(LOW)   |     0.0900 |
+---------+----------+------------+
| S(HIGH) | T(HEAVY) |     0.8100 |
+---------+----------+------------+


In [16]:
res = pgm.predict_state(['T', 'S'], evidence={'R': 'LIGHT_RAIN'})
res

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


{'T': 'HEAVY', 'S': 'LOW'}

In [2]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    print(vcfg)
    tm = VehicleManager(client=client, cfg=vcfg)

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg)
    try:
        world = client.get_world()

        tm.spawn_vehicles(number_of_vehicles=30)
        pm.spawn_walkers(number_of_walkers=pcfg.number_of_walkers)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)

In [3]:
try:
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config.yaml")
        print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

{'host': '127.0.0.1', 'port': 2000, 'asynch': True}
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'tm_port': 8000, 'safe': True, 'filterv': 'vehicle.*', 'generationv': 'All', 'hybrid': False, 'seed': 13, 'car_lights_on': False, 'hero': False, 'respawn': False, 'no_rendering': False}
You are currently in asynchronous mode. If this is a traffic simulation,             you could experience some issues. If it's not working correctly, switch to synchronous             mode by using self.traffic_manager.set_synchronous_mode(True)
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'number_of_walkers': 10, 'filterw': 'walker.pedestrian.*', 'generationw': '2', 'seedw': 0, 'running_percentage': 0.0, 'crossing_percentage': 0.0}

Added 30 vehicles
Spawned 10 walkers.
Sun(alt: -4.34, azm: 307.05) Storm(clouds=26%, rain=0%, wind=40%)            
Destroying 30 vehicles
Destroying 10 walkers

done.
